# Analyze Issue Script and Workflow Test

This notebook tests the `analyze_issue.py` script and the corresponding GitHub Actions workflow. The tests include validating the OpenAI API key, GitHub token, and running the script to check for errors.

In [ ]:
import os
import json
import openai
import requests
from dotenv import load_dotenv
from utils.openai_utils import parse_openai_response
from utils.github_utils import append_response_to_issue
from utils.analyze_issue import LLMIssueAnalyzer

# Load environment variables from .env file
load_dotenv()

# Configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_REPO = os.getenv("GITHUB_REPO", "https://github.com/pkuppens/my_chat_gpt")

# Set up OpenAI
openai.api_key = OPENAI_API_KEY

## Validate OpenAI API Key and Permissions

In [ ]:
openai_client = openai.OpenAI()

models = openai_client.get(path="models", cast_to=str)


In [ ]:
models_dict= json.loads(models)

In [ ]:

models_ids = [
    model["id"]
    for model in models_dict.get("data", [])
]

print(models_ids)

## Validate GitHub Token and Permissions

In [ ]:
def validate_github_token():
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    url = "https://api.github.com/user"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return True

if validate_github_token():
    print("GitHub token is valid.")
else:
    print("Invalid GitHub token or insufficient permissions.")

In [ ]:
assert GITHUB_REPO == 'https://github.com/pkuppens/my_chat_gpt', "Please set the GITHUB_REPO environment variable to your repository URL."

GITHUB_API_URL = GITHUB_REPO.replace("https://github.com", "https://api.github.com/repos")


In [ ]:
def get_github_issue(issue_number):
    url = f"{GITHUB_API_URL}/issues/{issue_number}"
    response = requests.get(url)
    response.raise_for_status()
    return response

response = get_github_issue(27)
print(response.text)

In [ ]:
issue_data = response.json()
title = issue_data["title"]
description = issue_data["body"]
print(f"Title: {title}")
print(f"Description: {description}")

In [ ]:
issue_data

In [ ]:
from utils.github_utils import ISSUE_TYPES, PRIORITY_LEVELS
from utils.prompts import load_analyze_issue_prompt

In [ ]:
placeholders = {
    "title": title,
    "description": description,
    "issue_types": ', '.join(ISSUE_TYPES),
    "priority_levels": ', '.join(PRIORITY_LEVELS),
    "issue_title": issue_data['title'],
    "issue_body": issue_data['body'],
    "issue_data": issue_data
}
placeholders

In [ ]:
system_prompt, user_prompt = load_analyze_issue_prompt(placeholders)
print(system_prompt)


In [ ]:
print(user_prompt)